In [1]:
import os
import json
import requests
from pybaseball import playerid_lookup, statcast_pitcher, pitching_stats

def get_player_data(first, last, year, img_dir="../img", data_dir="."):
    file_key = f"{first.lower()}_{last.lower()}"
    full_name = f"{first} {last}"

    df = playerid_lookup(last, first)
    if df.empty:
        raise ValueError(f"{full_name} not found")
    player_id = df["key_mlbam"].values[0]

    pitch_data = statcast_pitcher(start_dt=f"{year}-03-28", end_dt=f"{year}-10-01", player_id=player_id)
    pitch_csv_path = os.path.join(data_dir, f"{file_key}_{year}.csv")
    pitch_data.to_csv(pitch_csv_path, index=False)

    season_stats = pitching_stats(year, qual=50)
    player_stats = season_stats[season_stats['Name'] == full_name]
    stats_csv_path = os.path.join(data_dir, f"{file_key}_{year}_stats.csv")

    summary_cols = ['Name', 'Team', 'W', 'L', 'ERA', 'IP', 'WAR']
    player_stats[summary_cols].to_csv(stats_csv_path, index=False)

    img_url = f"https://img.mlbstatic.com/mlb-photos/image/upload/q_auto/v1/people/{player_id}/headshot/67/current.jpg"
    os.makedirs(img_dir, exist_ok=True)
    img_path = os.path.join(img_dir, f"{file_key}.jpg")
    res = requests.get(img_url)
    if res.status_code == 200:
        with open(img_path, "wb") as f:
            f.write(res.content)
    else:
        print(f"⚠️ Failed to download image for {full_name}")

    profile_url = f"https://statsapi.mlb.com/api/v1/people/{player_id}?hydrate=currentTeam"
    profile_res = requests.get(profile_url).json()
    person = profile_res["people"][0]
    team = person.get("currentTeam", {}).get("name", "N/A")
    number = person.get("primaryNumber", "N/A")

    return {
        "id": file_key,
        "name": full_name,
        "team": team,
        "number": f"No.{number}" if number != "N/A" else "N/A",
        "img": f"/img/{file_key}.jpg"
    }

failed_players = []

players = [
    ("Logan", "Gilbert"),
    ("Seth", "Lugo"),
    ("Logan", "Webb"),
    ("Zack", "Wheeler"),
    ("Aaron", "Nola"),
    ("Corbin", "Burnes"),
    ("Jose", "Berrios"),
    ("Tarik", "Skubal"),
    ("George", "Kirby"),
    ("Dylan", "Cease"),
    ("Jake", "Irvin"),
    ("Cole", "Ragans"),
    ("Pablo", "Lopez"),
    ("Kutter", "Crawford"),
    ("Luis", "Severino"),
    ("Cristopher", "Sanchez"),
    ("Sean", "Manaea"),
    ("Brandon", "Pfaadt"),
    ("Kevin", "Gausman"),
    ("JP", "Sears"),
    ("Bryce", "Miller"),
    ("Brady", "Singer"),
    ("Tyler", "Anderson"),
    ("Bailey", "Ober"),
    ("Tanner", "Houck"),
    ("Mitch", "Keller"),
    ("Chris", "Sale"),
    ("Erick", "Fedde"),
    ("Framber", "Valdez"),
    ("Yusei", "Kikuchi"),
    ("Luis", "Castillo"),
    ("Carlos", "Rodon"),
    ("Patrick", "Corbin"),
    ("Nestor", "Cortes"),
    ("Max", "Fried"),
    ("Freddy", "Peralta"),
    ("Tanner", "Bibee"),
    ("Michael", "King"),
    ("Shota", "Imanaga"),
    ("Miles", "Mikolas"),
    ("Griffin", "Canning"),
    ("Chris", "Bassitt"),
    ("Nathan", "Eovaldi"),
    ("Jose", "Quintana"),
    ("Hunter", "Brown"),
    ("Kyle", "Gibson"),
    ("Colin", "Rea"),
    ("Ronel", "Blanco"),
    ("Michael", "Wacha"),
    ("Sonny", "Gray"),
    ("MacKenzie", "Gore"),
    ("Charlie", "Morton"),
    ("Jameson", "Taillon"),
    ("Zach", "Eflin"),
    ("Austin", "Gomber"),
    ("Ryan", "Feltner"),
    ("Brayan", "Bello"),
    ("Jack", "Flaherty"),
    ("Aaron", "Civale"),
    ("Andrew", "Heaney"),
    ("Chris", "Flexen"),
    ("Zack", "Littell"),
    ("Marcus", "Stroman"),
    ("Luis", "Gil"),
    ("Mitch", "Spence"),
    ("Ben", "Lively"),
    ("Mitchell", "Parker"),
    ("Ryne", "Nelson"),
    ("Frankie", "Montas"),
    ("Ranger", "Suarez"),
    ("Hunter", "Greene"),
    ("Cal", "Quantrill"),
    ("Zac", "Gallen"),
    ("Javier", "Assad"),
    ("Matt", "Waldron"),
    ("Garrett", "Crochet"),
    ("Nick", "Pivetta"),
    ("Spencer", "Arrighetti"),
    ("Nick", "Martinez"),
    ("Bailey", "Falter"),
    ("Gavin", "Stone"),
    ("Taj", "Bradley"),
    ("Andrew", "Abbott"),
    ("Tobias", "Myers"),
    ("Reynaldo", "Lopez"),
    ("Luis", "L. Ortiz"),
    ("Joe", "Ryan"),
    ("Martin", "Perez"),
    ("Justin", "Steele"),
    ("Tyler", "Glasnow"),
    ("Albert", "Suarez"),
    ("Simeon", "Woods Richardson"),
    ("Paul", "Skenes"),
    ("Kyle", "Hendricks"),
    ("Michael", "Lorenzen"),
    ("Ryan", "Pepiot")
]

json_path = "players.json"
if os.path.exists(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        existing = json.load(f)
else:
    existing = []

processed_ids = {player['id'] for player in existing}
year = 2024

for i in range(0, len(players), 5):
    chunk = players[i:i+5]
    for first, last in chunk:
        file_key = f"{first.lower()}_{last.lower()}"
        if file_key in processed_ids:
            print(f"⏭️ {first} {last} already processed, skipping...")
            continue
        try:
            print(f"▶ {first} {last} processing...")
            player_json = get_player_data(first, last, year)
            existing.append(player_json)
            processed_ids.add(file_key)
        except Exception as e:
            print(f"❌ {first} {last} failed: {e}")
            failed_players.append((first, last)) 

    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(existing, f, indent=2, ensure_ascii=False)

print("Partial batch complete.")

if failed_players:
    print("\n Failed Players:")
    for first, last in failed_players:
        print(f"- {first} {last}")
else:
    print("\n All players processed successfully.")



KeyboardInterrupt: 